<a href="https://colab.research.google.com/github/mk7890/Resume-Parsing-System/blob/main/RESUME_PARSER_DATA_SCIENCE_FINAL_PROJECT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Necessary Libraries

In [ ]:
!pip install pymupdf pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 726.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

import fitz  # PyMuPDF
import pdfplumber

# Loading the Dataset

Mount Google Drive in Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.6 MB/s eta 0:00:00


# 1: Install Dependencies

In [ ]:
!pip install pdfminer.six spacy faker joblib pandas numpy scikit-learn torch transformers streamlit
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 110.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Python, NLP, and ML models to build a hybrid pipeline

# 2: Load & Extract Text from Resumes

In [ ]:
import os
import pdfplumber
import pandas as pd

folder_path = "/content/drive/MyDrive/resume_datasets_archive/data"

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()

# Read all resumes
resume_texts = []
resume_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

for file in resume_files:
    file_path = os.path.join(folder_path, file)
    text = extract_text_from_pdf(file_path)
    resume_texts.append({"filename": file, "text": text})

# Convert to DataFrame
resume_df = pd.DataFrame(resume_texts)
resume_df.to_csv("resume_texts.csv", index=False)
resume_df.head()


,filename,text
0,FINANCE (21).pdf,FINANCE AND OPERATIONS MANAGER\nExperience\nFi...
1,ENGINEERING (61).pdf,ENGINEERING TECHNICIAN\nSummary\nTo obtain a p...
2,FITNESS (24).pdf,"INTERN\nSummary\nMotivated, responsible Person..."
3,FINANCE (57).pdf,OPERATIONS FINANCE DIRECTOR\nSummary\nSkilled ...
4,DIGITAL_MEDIA (1).pdf,MEDIA ACTIVITIES SPECIALIST\nSummary\nMulti-Ta...


## 3: Check for Missing Values

In [ ]:
print("Missing Values:\n", resume_df.isnull().sum())


Missing Values:
 filename    0
text        0
dtype: int64


## 4: Fill Missing Values with Synthetic Data

used Faker to generate random names, emails, and phone numbers.

In [ ]:
from faker import Faker
import random

fake = Faker()

def fill_missing_values(df):
    for index, row in df.iterrows():
        if pd.isna(row["text"]) or len(row["text"].strip()) == 0:
            df.at[index, "text"] = fake.text()
    return df

resume_df = fill_missing_values(resume_df)
resume_df.to_csv("resume_cleaned.csv", index=False)
print("Missing values filled successfully!")

Missing values filled successfully!


# 5: Named Entity Recognition (NER) for Resume Fields
We'll use spaCy for extracting key information.

In [ ]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    name = ""
    email = ""
    phone = ""
    skills = []
    companies = []
    years_of_experience = ""

    # Extracting name
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
            break

    # Extracting email
    email_match = re.search(r"[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", text)
    if email_match:
        email = email_match.group(0)

    # Extracting phone number
    phone_match = re.search(r"\+?\d[\d -]{8,15}\d", text)
    if phone_match:
        phone = phone_match.group(0)

    # Extracting skills (Basic Skill Matching)
    common_skills = ["Python", "Machine Learning", "Data Science", "SQL", "NLP", "Deep Learning", "Java", "R", "Excel"]
    words = text.split()
    skills = list(set(words) & set(common_skills))

    # Extracting companies worked for (looking for organization names or company roles)
    company_keywords = ["Inc", "Ltd", "LLC", "Corporation", "Company"]
    companies = [sent.text for sent in doc.sents if any(keyword in sent.text for keyword in company_keywords)]

    # Extracting years of experience (searching for patterns like "5 years", "3 years of experience")
    experience_match = re.search(r"(\d+)\s*(years?|yrs?)\s*(experience|of\s*experience)", text, re.IGNORECASE)
    if experience_match:
        years_of_experience = experience_match.group(1)

    return {
        "name": name,
        "email": email,
        "phone": phone,
        "skills": skills,
        "companies": companies,
        "years_of_experience": years_of_experience
    }

# Assuming 'resume_df' is a DataFrame containing the resume text in a column called 'text'
resume_df["entities"] = resume_df["text"].apply(extract_entities)
resume_df.to_csv("resume_entities.csv", index=False)
resume_df.head()


,filename,text,entities
0,FINANCE (21).pdf,FINANCE AND OPERATIONS MANAGER\nExperience\nFi...,"{'name': 'Johnson', 'email': '', 'phone': '', ..."
1,ENGINEERING (61).pdf,ENGINEERING TECHNICIAN\nSummary\nTo obtain a p...,"{'name': '', 'email': '', 'phone': '', 'skills..."
2,FITNESS (24).pdf,"INTERN\nSummary\nMotivated, responsible Person...","{'name': 'Keep', 'email': '', 'phone': '', 'sk..."
3,FINANCE (57).pdf,OPERATIONS FINANCE DIRECTOR\nSummary\nSkilled ...,"{'name': 'SQL Strategic', 'email': '', 'phone'..."
4,DIGITAL_MEDIA (1).pdf,MEDIA ACTIVITIES SPECIALIST\nSummary\nMulti-Ta...,"{'name': 'Chattanooga State Created', 'email':..."


# 6: Job Role Classification
classify resumes into job roles.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from joblib import dump, load

# Sample labeled dataset
job_roles = ["Data Scientist", "Software Engineer", "Product Manager"]
resume_labels = [0, 1, 2]  # Assign labels (0=DS, 1=SE, 2=PM)

sample_resumes = [
    "Expert in Python, Machine Learning, and Deep Learning",
    "Experienced in Java, Software Development, and Agile",
    "Product strategy, market research, and stakeholder management"
]

# Train job classifier
vectorizer = TfidfVectorizer()
classifier = LogisticRegression()
pipeline = Pipeline([("tfidf", vectorizer), ("classifier", classifier)])

pipeline.fit(sample_resumes, resume_labels)
dump(pipeline, "job_classifier.pkl")

# Predict Job Role
resume_df["job_role"] = resume_df["text"].apply(lambda x: job_roles[pipeline.predict([x])[0]])
resume_df.to_csv("resume_classified.csv", index=False)


# 7: CV Rating based on Job Description Similarity
compare skills & experience with a given job description.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cv_score(cv_text, job_description):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([cv_text, job_description])
    score = cosine_similarity(vectors[0], vectors[1])[0][0]
    return round(score * 100, 2)

job_description = "Looking for a Data Scientist with experience in Python, NLP, and Machine Learning"
resume_df["cv_score"] = resume_df["text"].apply(lambda x: calculate_cv_score(x, job_description))
resume_df.to_csv("resume_scored.csv", index=False)


# 8: Save Processed Resumes as JSON
store the final structured resume data.

In [ ]:
import json

resume_json = resume_df.to_dict(orient="records")
with open("resume_parsed.json", "w") as f:
    json.dump(resume_json, f, indent=4)

print("Resumes saved as JSON!")


Resumes saved as JSON!


# 9: Deploy as a Streamlit App
build an interactive Streamlit app for real-time CV parsing.

In [ ]:
import streamlit as st

st.title("AI Resume Parser & CV Evaluator")

uploaded_file = st.file_uploader("Upload Resume (PDF)", type=["pdf"])

if uploaded_file:
    text = extract_text_from_pdf(uploaded_file)
    parsed_data = extract_entities(text)
    predicted_role = job_roles[pipeline.predict([text])[0]]
    cv_score = calculate_cv_score(text, job_description)

    st.write("### Extracted Details:")
    st.json(parsed_data)
    st.write(f"**Predicted Job Role:** {predicted_role}")
    st.write(f"**CV Score:** {cv_score}%")

    st.write("### CV Improvement Suggestions:")
    st.write("• Add missing skills from job description")
    st.write("• Include more detailed experience")

2025-02-06 19:39:17.414 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:17.518 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-06 19:39:17.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:17.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:17.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:17.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:17.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:17.523 Thread 'MainThread': mi

# CV OPTIMIZATION SUGGESTIONS

Analyze missing skills and recommend experience updates.

🔹 Code to Generate CV Optimization Suggestions

In [ ]:
def suggest_cv_improvements(cv_text, job_description):
    # Extract existing CV details
    extracted_data = extract_entities(cv_text)
    extracted_skills = set(extracted_data["skills"])

    # Extract keywords from job description
    job_words = job_description.split()
    job_skills = set([word for word in job_words if word.lower() in extracted_skills or word.lower().capitalize() in extracted_skills])

    missing_skills = job_skills - extracted_skills
    suggestions = []

    if missing_skills:
        suggestions.append(f"Consider adding these missing skills: {', '.join(missing_skills)}.")

    if not extracted_data["email"]:
        suggestions.append("Your resume is missing an email address.")

    if not extracted_data["phone"]:
        suggestions.append("Your resume is missing a phone number.")

    if not extracted_data["name"]:
        suggestions.append("Your resume is missing a name.")

    if not extracted_skills:
        suggestions.append("Your resume lacks technical keywords. Consider elaborating on your skills.")

    if len(cv_text.split()) < 150:
        suggestions.append("Your resume is too short. Add more details about your experience.")

    return suggestions

# Test Example
job_description = "Looking for a Data Scientist with experience in Python, NLP, and Machine Learning."
resume_text = resume_df["text"][0]  # Sample CV from dataset
cv_suggestions = suggest_cv_improvements(resume_text, job_description)

print("### CV Optimization Suggestions:")
for suggestion in cv_suggestions:
    print(f"- {suggestion}")


### CV Optimization Suggestions:
- Your resume is missing an email address.
- Your resume is missing a phone number.


# 11: Integrate CV Suggestions into Streamlit App
Display real-time improvement suggestions in our Streamlit app.

🔹 Update Streamlit App with CV Optimization

In [ ]:
import streamlit as st

st.title("AI Resume Parser & CV Evaluator")

uploaded_file = st.file_uploader("Upload Resume (PDF)", type=["pdf"])

if uploaded_file:
    text = extract_text_from_pdf(uploaded_file)
    parsed_data = extract_entities(text)
    predicted_role = job_roles[pipeline.predict([text])[0]]
    cv_score = calculate_cv_score(text, job_description)
    cv_suggestions = suggest_cv_improvements(text, job_description)

    st.write("### Extracted Details:")
    st.json(parsed_data)
    st.write(f"**Predicted Job Role:** {predicted_role}")
    st.write(f"**CV Score:** {cv_score}%")

    st.write("### CV Improvement Suggestions:")
    if cv_suggestions:
        for suggestion in cv_suggestions:
            st.write(f"✅ {suggestion}")
    else:
        st.write("🎉 Your CV is well-optimized for this job role!")


2025-02-06 19:39:37.837 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:37.838 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:37.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:37.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:37.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:37.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:39:37.843 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


# Enhancing Resume Optimization with AI-based Skill Matching, Readability Analysis, and Resume Formatting Improvements

## Step 12: AI-Based Skill Matching with BERT
Compare resume skills with job description skills using BERT embeddings for accurate similarity matching.

🔹 Install Required Packages

In [ ]:
!pip install sentence-transformers textstat


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load a pre-trained BERT model for similarity
model = SentenceTransformer("all-MiniLM-L6-v2")

def skill_matching(resume_text, job_description):
    # Extract skills from resume
    extracted_data = extract_entities(resume_text)
    resume_skills = set(extracted_data["skills"])

    # Extract skills from job description (naïve approach, ideally use NLP)
    job_skills = set(job_description.split())

    # Compute embeddings
    skill_embeddings = model.encode(list(resume_skills) + list(job_skills), convert_to_tensor=True)

    # Compute cosine similarity
    resume_skill_embeddings = skill_embeddings[:len(resume_skills)]
    job_skill_embeddings = skill_embeddings[len(resume_skills):]

    similarity_matrix = util.pytorch_cos_sim(resume_skill_embeddings, job_skill_embeddings)
    avg_similarity = torch.mean(similarity_matrix).item() * 100  # Convert to percentage

    return avg_similarity, resume_skills, job_skills

# Test Skill Matching
job_description = "Looking for a Data Scientist with Python, NLP, Machine Learning, Deep Learning."
resume_text = resume_df["text"][0]

similarity_score, resume_skills, job_skills = skill_matching(resume_text, job_description)
print(f"Skill Match Score: {similarity_score:.2f}%")


Skill Match Score: 18.83%


## 13: Grammar & Readability Analysis
Analyze grammar complexity, readability, and word count using textstat.

🔹 Code for Readability Analysis

In [ ]:
import textstat

def analyze_readability(cv_text):
    readability_score = textstat.flesch_reading_ease(cv_text)
    word_count = len(cv_text.split())
    grammar_complexity = textstat.text_standard(cv_text, float_output=True)

    feedback = []

    if readability_score < 50:
        feedback.append("Your resume is too complex. Consider simplifying the language.")
    elif readability_score > 70:
        feedback.append("Your resume is highly readable. Well done!")

    if word_count < 150:
        feedback.append("Your resume is too short. Add more details about your experience.")

    if grammar_complexity < 7:
        feedback.append("Your resume uses simple language. Consider using more professional terminology.")

    return feedback

# Test Readability Analysis
cv_text = resume_df["text"][0]
readability_feedback = analyze_readability(cv_text)
print("Readability Analysis Suggestions:")
for suggestion in readability_feedback:
    print(f"- {suggestion}")


Readability Analysis Suggestions:
- Your resume is too complex. Consider simplifying the language.


## 14: Automated Resume Formatting Improvements
Restructure unformatted resumes by:

✅ Adding bullet points for skills & experience

✅ Ensuring proper contact details placement

✅ Highlighting job-relevant sections

🔹 Code for Resume Formatting

In [ ]:
import re

def improve_resume_format(cv_text):
    # Ensure proper bullet points for skills
    cv_text = re.sub(r"(Skills:\s*)([^\n]+)", r"\1\n• \2", cv_text)

    # Ensure proper bullet points for experience
    cv_text = re.sub(r"(Experience:\s*)([^\n]+)", r"\1\n• \2", cv_text)

    # Highlight contact information
    cv_text = re.sub(r"(Email:\s*)([^\n]+)", r"\1📧 \2", cv_text)
    cv_text = re.sub(r"(Phone:\s*)([^\n]+)", r"\1📞 \2", cv_text)

    return cv_text

# Test Resume Formatting
formatted_resume = improve_resume_format(resume_df["text"][0])
print("Formatted Resume Preview:\n", formatted_resume[:1000])  # Display first 1000 chars

Formatted Resume Preview:
 FINANCE AND OPERATIONS MANAGER
Experience
Finance and Operations Manager , 07/2017 to Current
Company Name â€“ City , State
Brought on board to centralize progress in district.
Task with overseeing designated managers that monitor employee productivity, timely scheduling and provide updated trainings.
Institute policies, goals, objectives and procedures.
Challenge to construct and maintain effective cash flow monitoring system, review financial statements, audit sales commission and activity
reports.
Analyze performance data to measure productivity and steer continuous improvement initiatives with emphasis on recognizing cost streams
and reducing expenses.
Authorize various software platforms for employees.
Oversee inventory management, purchasing, and distribution.
Realign internal processes with introduction of inventory recording systems for the district.
Allocate monthly budget and create tentative forecast for proceeding month.
Financial Analyst , 07/201

## 15: Integrating Everything into Streamlit
Display AI-based skill matching, readability feedback, and improved formatting in Streamlit.

In [ ]:
import streamlit as st

st.title("AI Resume Parser & CV Optimizer")

uploaded_file = st.file_uploader("Upload Resume (PDF)", type=["pdf"])

if uploaded_file:
    text = extract_text_from_pdf(uploaded_file)
    parsed_data = extract_entities(text)
    predicted_role = job_roles[pipeline.predict([text])[0]]
    cv_score = calculate_cv_score(text, job_description)
    cv_suggestions = suggest_cv_improvements(text, job_description)
    skill_match_score, resume_skills, job_skills = skill_matching(text, job_description)
    readability_feedback = analyze_readability(text)
    formatted_resume = improve_resume_format(text)

    st.write("### Extracted Details:")
    st.json(parsed_data)
    st.write(f"**Predicted Job Role:** {predicted_role}")
    st.write(f"**CV Score:** {cv_score:.2f}%")
    st.write(f"**Skill Match Score:** {skill_match_score:.2f}%")

    st.write("### CV Optimization Suggestions:")
    if cv_suggestions:
        for suggestion in cv_suggestions:
            st.write(f"✅ {suggestion}")
    else:
        st.write("🎉 Your CV is well-optimized!")

    st.write("### Readability & Grammar Feedback:")
    if readability_feedback:
        for feedback in readability_feedback:
            st.write(f"📝 {feedback}")

    st.write("### Improved Resume Formatting:")
    st.text_area("Formatted Resume:", formatted_resume, height=250)


2025-02-06 19:40:47.053 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:40:47.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:40:47.055 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:40:47.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:40:47.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:40:47.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-06 19:40:47.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
import streamlit as st
import pdfplumber
import spacy
from collections import Counter
import matplotlib.pyplot as plt

# Load NLP model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(uploaded_file):
    """Extract text from an uploaded PDF file."""
    text = ""
    with pdfplumber.open(uploaded_file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def extract_keywords(text):
    """Extract keywords from text using Spacy."""
    doc = nlp(text.lower())
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return Counter(keywords)

def match_score(resume_keywords, job_keywords):
    """Calculate the matching score between resume and job description keywords."""
    resume_set = set(resume_keywords.keys())
    job_set = set(job_keywords.keys())
    common_words = resume_set.intersection(job_set)
    missing_words = job_set - resume_set
    score = len(common_words) / len(job_set) if job_set else 0
    return score * 100, common_words, missing_words

def plot_keyword_match(common, missing):
    """Plot common and missing keywords."""
    labels = ["Matched Keywords", "Missing Keywords"]
    sizes = [len(common), len(missing)]
    colors = ['#4CAF50', '#FF5733']
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
    ax.axis('equal')
    return fig

# Streamlit UI
st.title("Resume Matching App")

col1, col2 = st.columns(2)

with col1:
    uploaded_file = st.file_uploader("Upload Resume (PDF)", type="pdf")
    if uploaded_file:
        resume_text = extract_text_from_pdf(uploaded_file)
        st.text_area("Extracted Resume Text", resume_text, height=200)

with col2:
    job_description = st.text_area("Enter Job Description", height=200)

if uploaded_file and job_description:
    resume_keywords = extract_keywords(resume_text)
    job_keywords = extract_keywords(job_description)
    score, common_words, missing_words = match_score(resume_keywords, job_keywords)

    st.subheader("Match Score: {:.2f}%".format(score))
    st.pyplot(plot_keyword_match(common_words, missing_words))

    st.subheader("Missing Keywords:")
    st.write(", ".join(missing_words) if missing_words else "None")
